In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import  StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,LeakyReLU,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import Accuracy
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from concurrent.futures import ThreadPoolExecutor


In [30]:
data=pd.read_csv('../datasets/LFDE.csv').iloc[:,1:33]
labels=pd.read_csv('../datasets/Labels.csv').iloc[:,1:4]

In [31]:
data

,Fp1,AF3,F3,F7,FC5,FC1,C3,T7,CP5,CP1,...,FC2,Cz,C4,T8,CP6,CP2,P4,P8,PO4,O2
0,0.928540,0.929026,0.931885,0.932181,0.915036,0.913820,0.917995,0.955220,0.922876,0.895346,...,0.918463,0.908742,0.916730,0.941338,0.912970,0.892909,0.936960,0.930525,0.926157,0.919610
1,0.934599,0.930548,0.933244,0.956402,0.921351,0.912802,0.918702,0.958956,0.920417,0.898646,...,0.917011,0.907370,0.913715,0.941204,0.912695,0.893877,0.936948,0.935886,0.928080,0.925520
2,0.936406,0.938015,0.949734,0.958938,0.940361,0.928820,0.936064,0.963081,0.930597,0.901868,...,0.921571,0.911725,0.925427,0.951349,0.933269,0.899720,0.939760,0.939354,0.933353,0.932386
3,0.930833,0.927942,0.933852,0.939191,0.921358,0.915486,0.918335,0.963924,0.917324,0.897807,...,0.918805,0.917154,0.914547,0.947496,0.906814,0.896255,0.933491,0.913752,0.924676,0.920711
4,0.924605,0.925677,0.928969,0.930888,0.912942,0.907906,0.910938,0.964931,0.918882,0.893177,...,0.915207,0.909162,0.908083,0.949036,0.918089,0.890283,0.930360,0.937380,0.921526,0.917676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,0.949669,0.979452,0.943409,0.982937,0.975148,0.931089,0.969039,0.986993,0.980390,0.973023,...,0.994354,0.932102,0.972085,0.985957,0.986594,0.977557,0.966665,0.964311,0.986438,0.955828
1276,0.957463,0.978734,0.936155,0.978113,0.969349,0.929860,0.967728,0.975883,0.980747,0.964424,...,0.994287,0.924804,0.971189,0.984579,0.987108,0.978666,0.967171,0.964119,0.987002,0.963987
1277,0.950950,0.980199,0.944198,0.981367,0.971389,0.935194,0.970275,0.976879,0.981980,0.964400,...,0.994446,0.925746,0.968165,0.982839,0.987568,0.979365,0.968796,0.965780,0.987410,0.962439
1278,0.972825,0.981393,0.946748,0.978167,0.971954,0.933804,0.970888,0.976675,0.982886,0.964023,...,0.994950,0.922128,0.964904,0.980931,0.988455,0.980956,0.969704,0.964774,0.988463,0.963218


In [32]:
labels

,Valence,Arousal,Dominance
0,7.71,7.60,6.90
1,8.10,7.31,7.28
2,8.58,7.54,9.00
3,4.94,6.01,6.12
4,6.96,3.92,7.19
...,...,...,...
1275,3.91,6.96,5.82
1276,2.81,6.13,6.06
1277,3.05,7.01,5.10
1278,3.99,7.17,4.85


In [33]:
cols=['Fp1','F3','F7','C3','T7','CP5','P3','O1','Fp2','F8','T8']
#cols=data.columns
scaler=StandardScaler()
scaled_data=pd.DataFrame(scaler.fit_transform(data[cols]),columns=cols)

In [34]:
scaled_labels=labels>=5
encoder=LabelEncoder()
encoded_labels=[]
encoded_labels.append(encoder.fit_transform(scaled_labels.iloc[:,0]))# 0 for valence and 1 for arousal
encoded_labels=pd.DataFrame(np.array(encoded_labels).transpose(),columns=['label'])

In [35]:
scaled_data=scaled_data.to_numpy().reshape(1280,11)
encoded_labels = to_categorical(encoded_labels, num_classes=2)

In [36]:
def create_model():
    model1=Sequential()
    model1.add(Dense(256,activation=LeakyReLU(alpha=0.5),input_shape=(11,)))
    model1.add(Dropout(0.3))
    model1.add(Dense(256,activation=LeakyReLU(alpha=0.5)))
    model1.add(Dropout(0.3))
    model1.add(Dense(2,activation='softmax'))

    return model1

In [37]:
def train_and_evaluate(train_index,test_index,fold):
    X_train, X_test = scaled_data[train_index], scaled_data[test_index]
    y_train, y_test = encoded_labels[train_index], encoded_labels[test_index]
    model=create_model()
    model.compile(loss=categorical_crossentropy,optimizer=Adam(learning_rate=0.001),metrics=[Accuracy()])
    model.fit(X_train,y_train,epochs=150)
    model.save(f'../model_saves/MLP/LFDE/model_valence/model_thread_{fold}')
    result=model.predict(X_test)
    new_result=[]
    for i in (result):
        new_result.append(np.where(i==i.max())[0])
    new_result=np.array(new_result).flatten()
    new_y=[]
    for i in (y_test):
        new_y.append(np.where(i==i.max())[0])
    new_y=np.array(new_y).flatten()
    accuracy = accuracy_score(new_y,new_result)
    return accuracy

In [38]:
k_fold = KFold(n_splits=32,shuffle=False)
with ThreadPoolExecutor() as executor:
    futures = []
    fold=0
    for train_index, test_index in k_fold.split(scaled_data):
        fold+=1
        futures.append(executor.submit(train_and_evaluate, train_index, test_index, fold))
    results = [future.result() for future in futures]

print("Cross-validation accuracies:", results)
print("Mean accuracy:", np.mean(results))


Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
 4/39 [==>...........................] - ETA: 6s - loss: 0.7402 - accuracy: 0.0000e+0000WARNING:tensorflow:5 out of the last 15 calls to <function Model.make_train_function.<locals>.train_function at 0x0000023C5B25E340> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


38/39 [============================>.] - ETA: 0s - loss: 0.7361 - accuracy: 0.0000e+00Epoch 2/150
Epoch 2/150
 6/39 [===>..........................] - ETA: 0s - loss: 0.6863 - accuracy: 0.0000e+00Epoch 3/150
Epoch 3/150
20/39 [==============>...............] - ETA: 0s - loss: 0.7212 - accuracy: 0.0000e+00Epoch 4/150
Epoch 4/150
17/39 [============>.................] - ETA: 0s - loss: 0.6932 - accuracy: 0.0000e+00Epoch 5/150
Epoch 5/150
39/39 [==============================] - 1s 31ms/step - loss: 0.6986 - accuracy: 0.0000e+00
Epoch 6/150
 8/39 [=====>........................] - ETA: 0s - loss: 0.7014 - accuracy: 0.0000e+00Epoch 6/150
Epoch 7/150
39/39 [==============================] - 1s 23ms/step - loss: 0.6865 - accuracy: 0.0000e+00
Epoch 7/150
39/39 [==============================] - 1s 22ms/step - loss: 0.6810 - accuracy: 0.0000e+00
Epoch 7/150
10/39 [======>.......................] - ETA: 0s - loss: 0.6725 - accuracy: 0.0000e+00Epoch 8/150
Epoch 8/150
39/39 [=====================

INFO:tensorflow:Assets written to: MLP_LFDE/model_valence/model_thread_6\assets


14/39 [=========>....................] - ETA: 1s - loss: 0.6633 - accuracy: 0.0000e+00Epoch 145/150
Epoch 144/150
39/39 [==============================] - 3s 66ms/step - loss: 0.6465 - accuracy: 0.0000e+00


INFO:tensorflow:Assets written to: MLP_LFDE/model_valence/model_thread_1\assets


...

39/39 [==============================] - 0s 6ms/step - loss: 0.6475 - accuracy: 0.0000e+00
Epoch 143/150
39/39 [==============================] - 0s 5ms/step - loss: 0.6474 - accuracy: 0.0000e+00
Epoch 146/150
39/39 [==============================] - 0s 4ms/step - loss: 0.6484 - accuracy: 0.0000e+00
Epoch 144/150
39/39 [==============================] - 0s 3ms/step - loss: 0.6491 - accuracy: 0.0000e+00
Epoch 147/150
39/39 [==============================] - 0s 2ms/step - loss: 0.6471 - accuracy: 0.0000e+00
Epoch 145/150
39/39 [==============================] - 0s 2ms/step - loss: 0.6476 - accuracy: 0.0000e+00
Epoch 146/150
39/39 [==============================] - 0s 3ms/step - loss: 0.6498 - accuracy: 0.0000e+00
Epoch 149/150
39/39 [==============================] - 0s 3ms/step - loss: 0.6513 - accuracy: 0.0000e+00
Epoch 147/150
39/39 [==============================] - 0s 4ms/step - loss: 0.6624 - accuracy: 0.0000e+00
Epoch 150/150
39/39 [==============================] - 0s 4ms/step - 

INFO:tensorflow:Assets written to: MLP_LFDE/model_valence/model_thread_31\assets


2/2 [==============================] - 0s 22ms/step
INFO:tensorflow:Assets written to: MLP_LFDE/model_valence/model_thread_32\assets


INFO:tensorflow:Assets written to: MLP_LFDE/model_valence/model_thread_32\assets


2/2 [==============================] - 0s 2ms/step
Cross-validation accuracies: [0.5, 0.625, 0.475, 0.4, 0.425, 0.525, 0.7, 0.525, 0.4, 0.5, 0.45, 0.525, 0.525, 0.625, 0.55, 0.475, 0.575, 0.65, 0.575, 0.575, 0.525, 0.5, 0.525, 0.5, 0.525, 0.7, 0.475, 0.65, 0.475, 0.675, 0.35, 0.625]
Mean accuracy: 0.53515625
